In [1]:

import time
import pandas as pd
import re
import numpy as np
from konlpy.tag import Kkma, Okt
from sklearn.model_selection import train_test_split
import kss
import time
import os
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sn
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/home/bigdata/jupyter_notebook/wontae/NanumGothic.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location)
plt.rc('font', family = font_name.get_name())
%matplotlib inline

In [2]:
import os
import re
from keras import regularizers
import keras
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from keras import backend as K
from keras.models import Model
from keras import initializers
from keras.engine.topology import Layer
from keras.layers import Dense, Input, RepeatVector, Permute, Multiply, Lambda,BatchNormalization
from keras.layers import Embedding, GRU, Bidirectional, TimeDistributed, LSTM, Dropout
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils.np_utils import to_categorical
from nltk import tokenize
import tensorflow as tf
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences

import sentencepiece as spm


Using TensorFlow backend.
/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bigdata/.conda/envs/py36/l

In [3]:
class Attention(Layer):
    def __init__(self, regularizer = None, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.regularizer = regularizer
        self.supports_masking = True

    def build(self, input_shape):
        self.context = self.add_weight(name='context', shape=(input_shape[-1], 1),initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None),regularizer=self.regularizer,trainable=True)
        super(Attention, self).build(input_shape)
    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, x, mask=None):
        attention_in = K.exp(K.squeeze(K.dot(x, self.context), axis=-1))
        attention = attention_in/K.expand_dims(K.sum(attention_in, axis=-1), -1)

        if mask is not None:
        # use only the inputs specified by the mask
        # import pdb; pdb.set_trace()
            attention = attention*K.cast(mask, 'float32')

        weighted_sum = K.batch_dot(K.permute_dimensions(x, [0, 2, 1]), attention)
        return weighted_sum

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


In [4]:
! nvidia-smi

Wed Jul  1 10:06:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.37                 Driver Version: 396.37                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   29C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   27C    P0    32W / 250W |    323MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
petition_data = pd.read_csv('petition.csv', encoding='utf-8')

In [6]:
def title_content_combine(title,content):
    return title + ' ' + str(content)

In [7]:
petition_data['petition'] = petition_data.apply(lambda x : title_content_combine(x.title, x.content), axis = 1)

In [8]:
data = petition_data[['petition', 'category']]

In [9]:
max_nb_instances = 100000
x  = pd.DataFrame(data.petition)
y = pd.DataFrame(data.category)
y.columns = ['category']
x_train = x[:max_nb_instances]
y_train = y[:max_nb_instances]

In [10]:
kkma = Kkma()
okt = Okt()

In [11]:
def preprocess(text):
    text = text.replace('\n', '')
    t = re.sub('[^0-9a-zA-Zㄱ-힗,.!?""''\s]', '', text)
    return t

In [12]:
x['mod_petition'] = x.apply(lambda j : preprocess(j.petition), axis =1)

In [13]:
x_train = x.petition[:max_nb_instances]
y_train = pd.DataFrame(y_train)

In [14]:
from keras.preprocessing.text import Tokenizer
MAX_NB_WORDS = 100000

In [15]:
reviews = []
texts = []
for text in x.petition[:max_nb_instances]:
    sentences =kss.split_sentences(text)
    for sen in sentences:
        texts.append(sen)
    reviews.append(sentences)

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [17]:
MAX_SENTENCES = 10
MAX_SENTENCE_LENGTH = 20
X_train = np.zeros((max_nb_instances, MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTENCES:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENTENCE_LENGTH:
                    X_train[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

In [18]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 1241725 unique tokens.


In [19]:
def category_to_number(y_data):
    category_list = list(set(y_data.category))
    new_y = []
    for i, category in enumerate(y_data.category):
        new_y.append(category_list.index(category))
    ret = pd.DataFrame(new_y)
    ret.columns = ['category']
    return ret,category_list

In [20]:
y_train.columns = ['category']
Y_train,category_list = category_to_number(y_train)
Y_train = to_categorical(np.asarray(Y_train))

In [21]:

validation_split = 0.1
nb_validation_samples = int(validation_split * X_train.shape[0])

In [29]:
x_train = X_train[:-nb_validation_samples]
y_train = Y_train[:-nb_validation_samples]
x_val = X_train[-nb_validation_samples:]
y_val = Y_train[-nb_validation_samples:]

In [ ]:
result = []
for i, pet in enumerate(x.mod_petition[:max_nb_instances]):
    if i%5000==0: # 5,000의 배수로 While문이 실행될 때마다 몇 번째 While문 실행인지 출력.
        print("%d번째 for문."%i)
    tokenlist = okt.pos(pet, stem=True, norm=True) # 단어 토큰화
    temp=[]
    for word in tokenlist:
        if word[1] in ["Noun"]: # 명사일 때만
            temp.append((word[0])) # 해당 단어를 저장함

    if temp: # 만약 이번에 읽은 데이터에 명사가 존재할 경우에만
      result.append(temp) # 결과에 저장

In [ ]:
w2v_model = Word2Vec(result, size=embedding_dim, window=10, min_count=5, workers=4, sg=0)

In [ ]:
w2v_model.save('han_test_w2v-pre.model')

In [23]:
w2v_model = Word2Vec.load('han_test_w2v.model')
w2v_model = Word2Vec.load('han_test_w2v-pre.model')

In [24]:
def get_vector(word, word2vec_model):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

In [25]:
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index)+1, embedding_dim))
for word, i in tokenizer.word_index.items():
    temp = get_vector(word, w2v_model)
    if temp is not None:
        embedding_matrix[i] = temp

/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
l2_reg = regularizers.l2(1e-8)

sentence_in = Input(shape=(MAX_SENTENCE_LENGTH,), dtype='int32')
embedded_word_seq = Embedding(len(word_index) +1,embedding_dim, weights=[embedding_matrix], input_length = MAX_SENTENCE_LENGTH, trainable= True, mask_zero=True, name='word_embeddings',)(sentence_in)
word_encoder = Bidirectional(GRU(50, return_sequences=True, kernel_regularizer=l2_reg))(embedded_word_seq)
dense_transform_w = Dense(100, activation='relu', name='dense_transform_w', kernel_regularizer=l2_reg)(word_encoder)
attention_weighted_sentence = Model(sentence_in, Attention(name='word_attention', regularizer=l2_reg)(dense_transform_w))
word_attention_model = attention_weighted_sentence


texts_in = Input(shape=(MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
attention_weighted_sentences = TimeDistributed(attention_weighted_sentence)(texts_in)
sentence_encoder = Bidirectional(GRU(50, return_sequences=True, kernel_regularizer=l2_reg))(attention_weighted_sentences)
dense_transform_s = Dense(100, activation='relu', name='dense_transform_s',kernel_regularizer=l2_reg)(sentence_encoder)
attention_weighted_text = Attention(name='sentence_attention', regularizer=l2_reg)(dense_transform_s)
prediction = Dense(y_train.shape[1], activation='softmax')(attention_weighted_text)
model = Model(texts_in, prediction)

model.compile(optimizer=keras.optimizers.Adam(lr=0.001),loss='categorical_crossentropy',metrics=['acc'])


In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10, 20)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 100)           124228100 
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10, 100)           45300     
_________________________________________________________________
dense_transform_s (Dense)    (None, 10, 100)           10100     
_________________________________________________________________
sentence_attention (Attentio (None, 100)               100       
_________________________________________________________________
dense_1 (Dense)              (None, 17)                1717      
Total params: 124,285,317
Trainable params: 124,285,317
Non-trainable params: 0
______________________________________________________________

In [28]:

model.fit(x=[x_train],
          y=[y_train],
          batch_size=512,
          epochs=10,
          shuffle=True,
          validation_data=([x_val], 
                           [y_val]))

/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:109: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 124172600 elements. This may consume a large amount of memory.
  num_elements)
/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:109: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 124172600 elements. This may consume a large amount of memory.
  num_elements)


Train on 90000 samples, validate on 10000 samples
Epoch 1/10
90000/90000 [==============================] - 34s 375us/step - loss: 2.0448 - acc: 0.3572 - val_loss: 1.9656 - val_acc: 0.3891
Epoch 2/10
90000/90000 [==============================] - 25s 278us/step - loss: 1.3145 - acc: 0.5824 - val_loss: 1.8715 - val_acc: 0.4239
Epoch 3/10
90000/90000 [==============================] - 25s 279us/step - loss: 0.6337 - acc: 0.8090 - val_loss: 2.3008 - val_acc: 0.3893
Epoch 4/10
90000/90000 [==============================] - 25s 279us/step - loss: 0.3110 - acc: 0.9080 - val_loss: 2.7088 - val_acc: 0.3686
Epoch 5/10
90000/90000 [==============================] - 25s 276us/step - loss: 0.2066 - acc: 0.9364 - val_loss: 2.9960 - val_acc: 0.3622
Epoch 6/10
90000/90000 [==============================] - 25s 274us/step - loss: 0.1649 - acc: 0.9479 - val_loss: 3.1319 - val_acc: 0.3565
Epoch 7/10
90000/90000 [==============================] - 25s 274us/step - loss: 0.1429 - acc: 0.9541 - val_loss: 3.

In [30]:
def normalize(text):
    reviews = []
    texts = []
    return kss.split_sentences(text) 

def encode_input(reviews):
    
    
    
    ret = np.zeros((MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
    for j, sent in enumerate(reviews):
        if j < MAX_SENTENCES:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENTENCE_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                    ret[j, k] = tokenizer.word_index[word]
                    k = k + 1
    return ret

In [31]:
text1 = '공기업 비정규직의 정규화 그만해주세요. 만 하다가 처음으로 청원 글을 써봅니다. 공기업 비정규직의 정규직 전환 이제 그만해주십시오. 그간 한국도로공사 철도공사 서울교통공사 등등 많은 공기업들이 비정규직 정규화가 이루어졌습니다. 솔직히 비정규직 철폐라는 공약이 앞으로 비정규직 전형을 없애 채용하겠다던지, 해당 직렬의 자회사 정규직인 줄 알았습니다. 현실은 더 하더라구요.무분별한 비정규직의 정규화 당장 그만해야한다고 생각합니다.'
text2 = '6월 12일 오후 1시 발생한 사건입니다. 시장 한복판에서 길고양이 학대사건이 발생했습니다. 임신한 고양이가 매장에 들어 이유만으로 줄에 묶여 던지고 목을 조르고 던졌습니다. 그 결과 그 임신한 고양이는 피를 토하며 괴로워했고 상인이 고양이를 박스에 담아 합니다. 시장 주변에서 그 고양이를 담아간 상자를 발견했다고 하는데 그 고양이를 어찌 했는지 모르는 상황입니다. 시장 **가게 근처 **가게 골목 입니다. 정말 임신한 고양이가 매장에 이유 만으로 던지고 줄로 묶어 목을 조르고 이런 행동에서 인간다운 인간성이 있다는 생각이 드는지 의문입니다. 이런 행동을 하는 사람은 사회에 위협이 될 수 있다고 생각합니다. 정확한 수사를 통해 엄벌에 하는 바램입니다. 최근에 동물 학대 사건과 관련하여 이례적인 판례도 나왔습니다. 이를 발판으로 동물보호법 강화를 통해 생명을 경시하는 일이 발생하지 않았으면 하는 바램입니다.'

In [32]:
normalized_text = normalize(preprocess(text2))
encoded_text = encode_input(normalized_text)

hidden_word_encoding_out = Model(inputs=word_attention_model.input,outputs=word_attention_model.get_layer('dense_transform_w').output)
hidden_word_encodings = hidden_word_encoding_out.predict(encoded_text)
word_context = word_attention_model.get_layer('word_attention').get_weights()[0]
u_wattention = encoded_text*np.exp(np.squeeze(np.dot(hidden_word_encodings, word_context)))


reverse_word_index = {value:key for key,value in tokenizer.word_index.items()}



nopad_encoded_text = encoded_text[-len(text):]
nopad_encoded_text = [list(filter(lambda x: x > 0, sentence)) for sentence in nopad_encoded_text]
reconstructed_texts = [[reverse_word_index[int(i)] for i in sentence] for sentence in nopad_encoded_text]
nopad_wattention = u_wattention[:len(normalized_text)]
nopad_wattention = nopad_wattention/np.expand_dims(np.sum(nopad_wattention, -1), -1)
nopad_wattention = np.array([attention_seq[:len(sentence)] for attention_seq, sentence in zip(nopad_wattention, nopad_encoded_text)])
word_activation_maps = []


for i, att in enumerate(nopad_wattention):
    word_activation_maps.append(list(zip(reconstructed_texts[i], att)))

hidden_sentence_encoding_out = Model(inputs=model.input, outputs=model.get_layer('dense_transform_s').output)
hidden_sentence_encodings = np.squeeze(hidden_sentence_encoding_out.predict(np.expand_dims(encoded_text, 0)), 0)
sentence_context = model.get_layer('sentence_attention').get_weights()[0]
u_sattention = np.exp(np.squeeze(np.dot(hidden_sentence_encodings, sentence_context), -1))
nopad_sattention = u_sattention[-len(normalized_text):]

nopad_sattention = nopad_sattention/np.expand_dims(np.sum(nopad_sattention, -1), -1)

activation_map = list(zip(word_activation_maps, nopad_sattention))


In [33]:
activation_map

[([('6월', 0.04168116390776455),
   ('12일', 0.01932456377925637),
   ('오후', 0.0008100221776735624),
   ('1시', 0.6243790887451769),
   ('발생한', 0.21404099341761987),
   ('사건입니다', 0.0997641679725087)],
  0.12730905),
 ([('시장', 0.0019156067798971577),
   ('한복판에서', 0.5034924683751569),
   ('길고양이', 0.49417098075441435),
   ('발생했습니다', 0.0004209440905315677)],
  0.14146172),
 ([('임신한', 0.3506479820393029),
   ('고양이가', 0.45706276905742343),
   ('매장에', 0.12958215881449828),
   ('들어', 0.0002876960370728489),
   ('이유만으로', 0.0007544380184230185),
   ('묶여', 0.002569812492789212),
   ('던지고', 0.00965307341876743),
   ('목을', 0.012568667527527184),
   ('던졌습니다', 0.036873402594195664)],
  0.11549062),
 ([('그', 3.0514275381663843e-10),
   ('결과', 2.208520112826515e-07),
   ('그', 8.96776096155476e-09),
   ('임신한', 0.048438409394575194),
   ('고양이는', 0.45110201847139947),
   ('피를', 0.007482566760458755),
   ('고양이를', 0.08208902734610037),
   ('박스에', 0.3919827582617525),
   ('담아', 0.0189049764165638),
   ('합니다', 1

In [34]:
ans = model.predict(np.reshape(encoded_text, (1,)+encoded_text.shape))

In [35]:
ans.argsort()[0]

array([13, 10, 11,  1, 14,  3,  6, 15,  5,  4, 12, 16,  7,  2,  8,  0,  9])

In [36]:
category_list

['미래',
 '육아/교육',
 '반려동물',
 '경제민주화',
 '농산어촌',
 '행정',
 '보건복지',
 '기타',
 '교통/건축/국토',
 '성장동력',
 '정치개혁',
 '일자리',
 '저출산/고령화대책',
 '외교/통일/국방',
 '인권/성평등',
 '문화/예술/체육/언론',
 '안전/환경']

In [ ]:
yahoo = pd.read_csv('yahoo.csv')

In [ ]:
yahoo_x= pd.DataFrame(yahoo.ko)
yahoo_y = pd.DataFrame(yahoo.label)

In [ ]:
yahoo_reviews = []
yahoo_texts = []
for text in yahoo_x.ko:
    sentences =kss.split_sentences(text)
    for sen in sentences:
        yahoo_texts.append(sen)
    yahoo_reviews.append(sentences)

In [ ]:
tokenizer.fit_on_texts(yahoo_texts)
MAX_SENTENCES = 10
MAX_SENTENCE_LENGTH = 50
yahoo_X_train = np.zeros((max_nb_instances, MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
for i, sentences in enumerate(yahoo_reviews):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTENCES:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENTENCE_LENGTH:
                    X_train[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

In [ ]:
yahoo_Y_train = to_categorical(np.asarray(yahoo_y))

validation_split = 0.1
nb_validation_samples = int(validation_split * yahoo_X_train.shape[0])

yahoo_x_train = yahoo_X_train[:-nb_validation_samples]
yahoo_y_train = yahoo_Y_train[:-nb_validation_samples]
yahoo_x_val = yahoo_X_train[-nb_validation_samples:]
yahoo_y_val = yahoo_Y_train[-nb_validation_samples:]

In [ ]:
l2_reg = regularizers.l2(1e-8)

sentence_in = Input(shape=(MAX_SENTENCE_LENGTH,), dtype='int32')
embedded_word_seq = Embedding(len(word_index) +1,embedding_dim, weights=[embedding_matrix], input_length = MAX_SENTENCE_LENGTH, trainable= True, mask_zero=True, name='word_embeddings',)(sentence_in)
word_encoder = Bidirectional(GRU(50, return_sequences=True, kernel_regularizer=l2_reg))(embedded_word_seq)
dense_transform_w = Dense(100, activation='relu', name='dense_transform_w', kernel_regularizer=l2_reg)(word_encoder)
attention_weighted_sentence = Model(sentence_in, Attention(name='word_attention', regularizer=l2_reg)(dense_transform_w))
word_attention_model = attention_weighted_sentence


texts_in = Input(shape=(MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
attention_weighted_sentences = TimeDistributed(attention_weighted_sentence)(texts_in)
sentence_encoder = Bidirectional(GRU(50, return_sequences=True, kernel_regularizer=l2_reg))(attention_weighted_sentences)
dense_transform_s = Dense(100, activation='relu', name='dense_transform_s',kernel_regularizer=l2_reg)(sentence_encoder)
attention_weighted_text = Attention(name='sentence_attention', regularizer=l2_reg)(dense_transform_s)
yahoo_prediction = Dense(yahoo_y_train.shape[1], activation='softmax')(attention_weighted_text)
yahoo_model = Model(texts_in, yahoo_prediction)

yahoo_model.compile(optimizer=keras.optimizers.Adam(lr=0.001),loss='categorical_crossentropy',metrics=['acc'])

In [ ]:

yahoo_model.fit(x=[yahoo_x_train],
          y=[yahoo_y_train],
          batch_size=512,
          epochs=10,
          shuffle=True,
          validation_data=([yahoo_x_val], 
                           [yahoo_y_val]))

In [ ]:
w2v_model2 = Word2Vec.load('ko.bin')

In [ ]:
embedding_dim = 200
embedding_matrix = np.zeros((len(word_index)+1, embedding_dim))
for word, i in tokenizer.word_index.items():
    temp = get_vector(word, w2v_model2)
    if temp is not None:
        embedding_matrix[i] = temp